In [ ]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats


In [ ]:
# Clean Dataset

# Reads CSV file and and delete rows
df = pd.read_csv('raw_data.csv', skiprows = 9)

# Delete the last 22 rows
df = df.iloc[:-22]

# Clean column names
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)

# Delete columns not relevant to the report
df = df.drop(columns=[
    "International travellers entering or returning to Canada",
    "Residents of countries other than the United States of America, air",
    "Residents of countries other than the United States of America, land",
    "Residents of countries other than the United States of America, water",
    "Canadian-resident visitors returning to Canada"
], errors="ignore")

# Function to parse mixed date formats
def fix_reference_period(date_str):
    try:
        if "-" in date_str:
            parts = date_str.split("-")

            if parts[0].isdigit():
                year = int(parts[0]) + 2000
                month = parts[1]
            else:
                month = parts[0]
                year = int(parts[1])
                if year < 100:
                    year += 2000

            # Convert to datetime and format as YYYY-MM
            return pd.to_datetime(f"{month} {year}", format="%b %Y").strftime("%Y-%m")

   # Debugging and blank value caveat
    except Exception as e:
        print(f"Error converting {date_str}: {e}")
        return None

# Apply function to fix dates
df["Reference period"] = df["Reference period"].apply(fix_reference_period)

# Change data to float values
num_cols = df.columns[2:]
df[num_cols] = df[num_cols].replace(",", "", regex=True).astype(float)

# Apply relevant traveller type to all rows
df["Traveller type"] = df["Traveller type"].ffill()

# Save cleaned data as a new CSV file
df.to_csv('clean_data.csv', index = False)

NameError: name 'pd' is not defined

In [ ]:
# Two-tail t-test for Canadians Returning from the US vs Americans Entering Canada (Annually)

# Filter rows where the 'Traveller type' is 'Travellers'
df_filtered = df[df['Traveller type'] == 'Travellers'].copy()

# Convert 'Reference period' to datetime format
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')

# Check for any none time values after conversion
if df_filtered['Reference period'].isna().sum() > 0:
    print(f"There are {df_filtered['Reference period'].isna().sum()} invalid date entries.")
    print(df_filtered[df_filtered['Reference period'].isna()])

# Extract the year from the 'Reference period' column
df_filtered['Year'] = df_filtered['Reference period'].dt.year

# Aggregate the data by year
annual_data = df_filtered.groupby('Year').agg({
    'United States of America residents entering Canada': 'sum',
    'Canadian residents returning from the United States of America': 'sum'
}).reset_index()

# Create variables for the t-test based on the annual aggregated data
us_residents_entering_canada = annual_data['United States of America residents entering Canada']
canadian_residents_returning_usa = annual_data['Canadian residents returning from the United States of America']

# Run the t-test
t_statistic, p_value = stats.ttest_ind(us_residents_entering_canada, canadian_residents_returning_usa)

# Calculate the means of both groups
mean_us_entering_canada = us_residents_entering_canada.mean()
mean_canadians_returning_usa = canadian_residents_returning_usa.mean()

# Print the means
print(f'Mean of U.S. residents entering Canada: {mean_us_entering_canada}')
print(f'Mean of Canadian residents returning from the U.S.: {mean_canadians_returning_usa}')

# Print the results of the t-test
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference between U.S. residents entering Canada and Canadian residents returning to the U.S.")
else:
    print("There is no significant difference between U.S. residents entering Canada and Canadian residents returning to the U.S.")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Two tail t-test for Excursionist vs Tourist Canadians Returning from the US (Annually)

# Clear the object
df_filtered = df.copy()

# Filter rows where the 'Traveller type' is 'Excursionists' or 'Tourists'
df_filtered = df[df['Traveller type'].isin(['Excursionists', 'Tourists'])].copy()

# Convert 'Reference period' to datetime format
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')

# Check for any none time values after conversion
if df_filtered['Reference period'].isna().sum() > 0:
    print(f"There are {df_filtered['Reference period'].isna().sum()} invalid date entries.")
    print(df_filtered[df_filtered['Reference period'].isna()])

# Extract the year from the 'Reference period' column
df_filtered['Year'] = df_filtered['Reference period'].dt.year

# Filter for Canadian residents returning from the U.S. based on Excursionists and Tourists
df_canadian_return_excursionists = df_filtered[(df_filtered['Traveller type'] == 'Excursionists') &
                                               df_filtered['Canadian residents returning from the United States of America'].notna()]

df_canadian_return_tourists = df_filtered[(df_filtered['Traveller type'] == 'Tourists') &
                                          df_filtered['Canadian residents returning from the United States of America'].notna()]

# Aggregate the data by year for both Excursionists and Tourists
annual_excursionists = df_canadian_return_excursionists.groupby('Year').agg({
    'Canadian residents returning from the United States of America': 'sum'
}).reset_index()

annual_tourists = df_canadian_return_tourists.groupby('Year').agg({
    'Canadian residents returning from the United States of America': 'sum'
}).reset_index()

# Create variables for the t-test
canadian_residents_returning_excursionists = annual_excursionists['Canadian residents returning from the United States of America']
canadian_residents_returning_tourists = annual_tourists['Canadian residents returning from the United States of America']

# Calculate the means for both groups
mean_canadian_residents_returning_excursionists = canadian_residents_returning_excursionists.mean()
mean_canadian_residents_returning_tourists = canadian_residents_returning_tourists.mean()

# Print mean results
print(f'Mean of Canadian Excursionists Returning from the U.S.: {mean_canadian_residents_returning_excursionists}')
print(f'Mean of Canadian Tourists Returning from the U.S.: {mean_canadian_residents_returning_tourists}')

# Run the t-test
t_statistic, p_value = stats.ttest_ind(canadian_residents_returning_excursionists, canadian_residents_returning_tourists)

# Print the results of the t-test
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference between Excursionists and Tourists for Canadian residents returning from the U.S.")
else:
    print("There is no significant difference between Excursionists and Tourists for Canadian residents returning from the U.S.")


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Two tail t-test for Excursionist vs Tourist Americans Entering Canada (Annually)

# Clear object
df_filtered = df.copy()

# Filter rows where the 'Traveller type' is 'Excursionists' or 'Tourists'
df_filtered = df[df['Traveller type'].isin(['Excursionists', 'Tourists'])].copy()

# Convert 'Reference period' to datetime format
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')

# Extract the year from the 'Reference period' column
df_filtered['Year'] = df_filtered['Reference period'].dt.year

# Filter for American residents entering Canada based on Excursionists and Tourists
df_us_residents_entering_canada_excursionists = df_filtered[(df_filtered['Traveller type'] == 'Excursionists') &
                                                           df_filtered['United States of America residents entering Canada'].notna()]

df_us_residents_entering_canada_tourists = df_filtered[(df_filtered['Traveller type'] == 'Tourists') &
                                                      df_filtered['United States of America residents entering Canada'].notna()]

# Aggregate the data by year for both Excursionists and Tourists
annual_excursionists = df_us_residents_entering_canada_excursionists.groupby('Year').agg({
    'United States of America residents entering Canada': 'sum'
}).reset_index()

annual_tourists = df_us_residents_entering_canada_tourists.groupby('Year').agg({
    'United States of America residents entering Canada': 'sum'
}).reset_index()

# Create variables for t-test
us_residents_entering_canada_excursionists = annual_excursionists['United States of America residents entering Canada']
us_residents_entering_canada_tourists = annual_tourists['United States of America residents entering Canada']

# Calculate the means for both groups
mean_us_residents_entering_canada_excursionists = us_residents_entering_canada_excursionists.mean()
mean_us_residents_entering_canada_tourists = us_residents_entering_canada_tourists.mean()

# Print mean results
print(f'Mean of American Excursionists Entering Canada: {mean_us_residents_entering_canada_excursionists}')
print(f'Mean of American Tourists Entering Canada: {mean_us_residents_entering_canada_tourists}')

# Run t-test
t_statistic, p_value = stats.ttest_ind(us_residents_entering_canada_excursionists, us_residents_entering_canada_tourists)

# Print the results of the t-test
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference between Excursionists and Tourists for U.S. residents entering Canada.")
else:
    print("There is no significant difference between Excursionists and Tourists for U.S. residents entering Canada.")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Two-way ANOVA for Excursionists and Tourists for Canadian residents returning from the U.S. and U.S. residents entering Canada (Annually)

# Clear object
df_filtered = df.copy()

# Filter rows where the 'Traveller type' is 'Excursionists' or 'Tourists'
df_filtered = df[df['Traveller type'].isin(['Excursionists', 'Tourists'])].copy()

# Convert 'Reference period' to datetime format
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')

# Extract the year from the 'Reference period' column
df_filtered['Year'] = df_filtered['Reference period'].dt.year

# Aggregate the data by year for both groups (Canadians and Americans)
annual_data = df_filtered.groupby(['Year', 'Traveller type']).agg({
    'United States of America residents entering Canada': 'sum',
    'Canadian residents returning from the United States of America': 'sum'
}).reset_index()

# Combine the two columns (Canadian and U.S. residents) into one column for the ANOVA
# Create a new column indicating 'Travel Direction' (either returning Canadians or entering Americans)
annual_data_melted = annual_data.melt(id_vars=['Year', 'Traveller type'], value_vars=[
    'United States of America residents entering Canada',
    'Canadian residents returning from the United States of America'
], var_name='Travel Direction', value_name='Travelers')

# Run a two-way ANOVA
f_statistic, p_value = stats.f_oneway(
    annual_data_melted[annual_data_melted['Traveller type'] == 'Excursionists']['Travelers'],
    annual_data_melted[annual_data_melted['Traveller type'] == 'Tourists']['Travelers']
)

# Print the results of the ANOVA
print(f'F-statistic: {f_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference between Excursionists and Tourists for Canadian residents returning from the U.S. and U.S. residents entering Canada.")
else:
    print("There is no significant difference between Excursionists and Tourists for Canadian residents returning from the U.S. and U.S. residents entering Canada.")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# ANOVA of Entry Type (air, land, water) of Americans Entering Canada (Annually)

# Filter rows where the 'Traveller type' is 'Travellers' and the 'Reference period' is for U.S. residents entering Canada
df_filtered = df.copy()
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')
df_us_residents = df_filtered[(df_filtered['Traveller type'] == 'Travellers') &
                              (df_filtered['United States of America residents, air'].notna())].copy()

# Create a new column for the year by extracting from 'Reference period'
df_us_residents['Year'] = df_us_residents['Reference period'].dt.year

# Aggregate the data by year and sum the travel counts for each entry type (Air, Land, Water)
annual_entry_data = df_us_residents.groupby('Year')[[
    'United States of America residents, air',
    'United States of America residents, land',
    'United States of America residents, water']].sum().reset_index()

# Perform one-way ANOVA for entry types (Air, Land, Water)
f_statistic, p_value = stats.f_oneway(
    annual_entry_data['United States of America residents, air'],
    annual_entry_data['United States of America residents, land'],
    annual_entry_data['United States of America residents, water']
)

# Print the results
print(f'F-statistic: {f_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference in annual visits by entry type (Air, Land, Water).")
else:
    print("There is no significant difference in annual visits by entry type (Air, Land, Water).")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# ANOVA of Entry Type (air, land, water) of Canadians Returning from the US (Annually)

# Filter rows where the 'Traveller type' is 'Travellers' and the 'Reference period' is for Canadian residents returning from the U.S.
df_filtered = df.copy()
df['Reference period'] = pd.to_datetime(df['Reference period'], errors='coerce')
df_filtered['Reference period'] = pd.to_datetime(df_filtered['Reference period'], errors='coerce')
df_canadian_residents = df_filtered[(df_filtered['Traveller type'] == 'Travellers') &
                                    (df_filtered['Canadian residents returning from the United States of America, air'].notna())].copy()

# Create a new column for the year by extracting from 'Reference period'
df_canadian_residents['Year'] = df_canadian_residents['Reference period'].dt.year

# Aggregate the data by year and sum the travel counts for each entry type (Air, Land, Water)
annual_entry_data_canadians = df_canadian_residents.groupby('Year')[[
    'Canadian residents returning from the United States of America, air',
    'Canadian residents returning from the United States of America, land',
    'Canadian residents returning from the United States of America, water']].sum().reset_index()

# Perform one-way ANOVA for entry types (Air, Land, Water) for Canadian residents
f_statistic, p_value = stats.f_oneway(
    annual_entry_data_canadians['Canadian residents returning from the United States of America, air'],
    annual_entry_data_canadians['Canadian residents returning from the United States of America, land'],
    annual_entry_data_canadians['Canadian residents returning from the United States of America, water']
)

# Print the results
print(f'F-statistic: {f_statistic}')
print(f'P-value: {p_value}')

# Interpretation of results
if p_value < 0.05:
    print("There is a significant difference in annual visits by entry type (Air, Land, Water) for Canadian residents.")
else:
    print("There is no significant difference in annual visits by entry type (Air, Land, Water) for Canadian residents.")

TypeError: unsupported operand type(s) for /: 'str' and 'int'